# AWS Postgres RDS Debug Notebook

This notebook allows to run queries manually against the AWS postgres DB.

In [6]:
import sys
sys.path.insert(0, '/Users/ferdi/Documents/agent-copilot/src')

import psycopg2
import psycopg2.extras
from urllib.parse import urlparse
import pandas as pd
from datetime import datetime
import os

# Import the database module
from aco.server import db

## Check Database Connection

In [7]:
# EMERGENCY NETWORK DIAGNOSIS - Run this first!
print("🚨 Basic network connectivity test...")

import socket
import subprocess
from urllib.parse import urlparse
from aco.common.constants import DATABASE_URL

if DATABASE_URL:
    parsed = urlparse(DATABASE_URL)
    host = parsed.hostname
    port = parsed.port or 5432
    
    print(f"Target: {host}:{port}")
    
    # Test 1: DNS resolution (should be instant)
    print(f"\n1️⃣ Testing DNS resolution for {host}...")
    try:
        import socket
        ip = socket.gethostbyname(host)
        print(f"✅ DNS resolved to: {ip}")
    except Exception as e:
        print(f"❌ DNS resolution failed: {e}")
        print("→ This means the hostname is invalid or DNS is broken")
    
    # Test 2: Ping test (basic connectivity)
    print(f"\n2️⃣ Testing ping to {host}...")
    try:
        result = subprocess.run(['ping', '-c', '2', host], 
                              capture_output=True, text=True, timeout=10)
        if result.returncode == 0:
            print("✅ Ping successful")
            # Extract timing info
            lines = result.stdout.split('\n')
            for line in lines:
                if 'time=' in line:
                    print(f"   {line.strip()}")
        else:
            print("❌ Ping failed")
            print(f"   Error: {result.stderr}")
    except subprocess.TimeoutExpired:
        print("❌ Ping timeout - host unreachable")
    except Exception as e:
        print(f"❌ Ping error: {e}")
    
    # Test 3: Telnet-style connection test (fastest way to test port)
    print(f"\n3️⃣ Testing port connectivity to {host}:{port}...")
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(5)  # Very short timeout
        result = sock.connect_ex((host, port))
        sock.close()
        
        if result == 0:
            print("✅ Port is reachable")
        else:
            print(f"❌ Port connection failed (error {result})")
            print("   → Database server may be down or port blocked")
    except Exception as e:
        print(f"❌ Socket error: {e}")
        
else:
    print("❌ No DATABASE_URL found")

# Test 4: General internet connectivity 
print(f"\n4️⃣ Testing general internet connectivity...")
try:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(3)
    result = sock.connect_ex(("8.8.8.8", 53))  # Google DNS
    sock.close()
    
    if result == 0:
        print("✅ Internet connectivity OK")
    else:
        print("❌ No internet connectivity")
        print("   → Check your network connection")
except Exception as e:
    print(f"❌ Internet test failed: {e}")

print(f"\n🔍 If all tests pass but db.get_conn() hangs:")
print("   → The PostgreSQL server is likely overloaded or in maintenance")
print("   → Check AWS RDS console for instance status")
print("   → Try connecting from a different network/location")

🚨 Basic network connectivity test...
Target: workflow-postgres.cm14iy6021bi.us-east-1.rds.amazonaws.com:5432

1️⃣ Testing DNS resolution for workflow-postgres.cm14iy6021bi.us-east-1.rds.amazonaws.com...
✅ DNS resolved to: 98.90.57.89

2️⃣ Testing ping to workflow-postgres.cm14iy6021bi.us-east-1.rds.amazonaws.com...
❌ Ping timeout - host unreachable

3️⃣ Testing port connectivity to workflow-postgres.cm14iy6021bi.us-east-1.rds.amazonaws.com:5432...
✅ Port is reachable

4️⃣ Testing general internet connectivity...
✅ Internet connectivity OK

🔍 If all tests pass but db.get_conn() hangs:
   → The PostgreSQL server is likely overloaded or in maintenance
   → Check AWS RDS console for instance status
   → Try connecting from a different network/location


In [8]:
# Enhanced database connection debugging
from aco.common.constants import DATABASE_URL
import psycopg2
from urllib.parse import urlparse

if DATABASE_URL:
    print(f"✅ Database URL found: {DATABASE_URL.split('@')[1] if '@' in DATABASE_URL else DATABASE_URL}")
    
    # Parse the URL to show connection details
    try:
        parsed = urlparse(DATABASE_URL)
        print(f"\n📋 Connection details:")
        print(f"   Host: {parsed.hostname}")
        print(f"   Port: {parsed.port or 5432}")
        print(f"   Database: {parsed.path[1:] if parsed.path else 'N/A'}")
        print(f"   Username: {parsed.username}")
        print(f"   Password: {'***' if parsed.password else 'None'}")
    except Exception as e:
        print(f"❌ Error parsing DATABASE_URL: {e}")
    
    # Test raw psycopg2 connection first
    print(f"\n🔍 Testing raw psycopg2 connection...")
    try:
        result = urlparse(DATABASE_URL)
        raw_conn = psycopg2.connect(
            host=result.hostname,
            port=result.port or 5432,
            user=result.username,
            password=result.password,
            database=result.path[1:],
            connect_timeout=30
        )
        print("✅ Raw psycopg2 connection successful")
        raw_conn.close()
    except psycopg2.OperationalError as e:
        print(f"❌ psycopg2 connection failed: {e}")
        print("   This could be:")
        print("   - Network connectivity issue")
        print("   - Database server down")
        print("   - Authentication failure")
        print("   - Firewall blocking connection")
    except Exception as e:
        print(f"❌ Unexpected connection error: {e}")
    
    # Test db.get_conn() function
    print(f"\n🔍 Testing db.get_conn() function...")
    try:
        conn = db.get_conn()
        print("✅ db.get_conn() successful")
        print("✅ Connected to database successfully")
    except Exception as e:
        print(f"❌ db.get_conn() failed: {e}")
        print(f"   Error type: {type(e).__name__}")
        
        # Check if it's a timeout
        if "timeout" in str(e).lower():
            print("   → This looks like a timeout issue")
        elif "authentication" in str(e).lower() or "password" in str(e).lower():
            print("   → This looks like an authentication issue")
        elif "connection refused" in str(e).lower():
            print("   → This looks like the server is not accepting connections")
        elif "host" in str(e).lower():
            print("   → This looks like a hostname resolution issue")
            
else:
    print("❌ No DATABASE_URL found")
    print("   Check your environment variables or config.yaml")

✅ Database URL found: workflow-postgres.cm14iy6021bi.us-east-1.rds.amazonaws.com:5432/workflow_db

📋 Connection details:
   Host: workflow-postgres.cm14iy6021bi.us-east-1.rds.amazonaws.com
   Port: 5432
   Database: workflow_db
   Username: postgres
   Password: ***

🔍 Testing raw psycopg2 connection...
✅ Raw psycopg2 connection successful

🔍 Testing db.get_conn() function...
✅ db.get_conn() successful
✅ Connected to database successfully


## List Table Entries

### experiments table

In [10]:
# Get all experiments
experiments = db.query_all(
    "SELECT session_id, parent_session_id, name, timestamp, success, notes FROM experiments ORDER BY timestamp DESC LIMIT 20"
)

if experiments:
    df_experiments = pd.DataFrame(experiments)
    print(f"Found {len(experiments)} experiments:")
    display(df_experiments)
else:
    print("No experiments found in database")

2025-11-17 02:34:36,496 - ACO - WARNING - Database connection error on attempt 1: SSL connection has been closed unexpectedly

2025-11-17 02:34:37,109 - ACO - DEBUG - Database schema initialized
2025-11-17 02:34:37,111 - ACO - INFO - Initialized PostgreSQL connection to workflow-postgres.cm14iy6021bi.us-east-1.rds.amazonaws.com


No experiments found in database


### llm_calls table

In [12]:
# Get recent LLM calls
llm_calls = db.query_all(
    "SELECT session_id, node_id, api_type, timestamp FROM llm_calls ORDER BY timestamp DESC LIMIT 20"
)

if llm_calls:
    df_llm = pd.DataFrame(llm_calls)
    print(f"Found {len(llm_calls)} recent LLM calls:")
    display(df_llm)
else:
    print("No LLM calls found")

No LLM calls found


# Drop tables and change schema (DANGER ZONE)

In [ ]:
# FRESH CONNECTION + EMERGENCY RESET (All-in-one)
print("🚨 Creating fresh connection and performing emergency reset...")
from aco.common.constants import DATABASE_URL
import psycopg2
import psycopg2.extras
from urllib.parse import urlparse

if DATABASE_URL:
    parsed = urlparse(DATABASE_URL)
    
    try:
        # Create a fresh connection
        print("1️⃣ Establishing fresh connection...")
        fresh_conn = psycopg2.connect(
            host=parsed.hostname,
            port=parsed.port or 5432,
            user=parsed.username,
            password=parsed.password,
            database=parsed.path[1:],
            connect_timeout=15
        )
        fresh_conn.autocommit = True  # Auto-commit for immediate effect
        cursor = fresh_conn.cursor()
        print("   ✅ Fresh connection established")
        
        # Kill all other connections
        print("\n2️⃣ Killing hanging sessions...")
        cursor.execute("SELECT current_database()")
        current_db = cursor.fetchone()[0]
        
        cursor.execute("""
            SELECT pg_terminate_backend(pid) as killed
            FROM pg_stat_activity
            WHERE datname = %s 
            AND pid != pg_backend_pid()
            AND state != 'idle'
        """, (current_db,))
        killed_sessions = cursor.fetchall()
        print(f"   ✅ Terminated {len(killed_sessions)} hanging sessions")
        
        # Force drop all tables (uncomment below)
        # print("\n3️⃣ Force dropping tables...")
        # cursor.execute("DROP TABLE IF EXISTS llm_calls CASCADE")
        # cursor.execute("DROP TABLE IF EXISTS attachments CASCADE")
        # cursor.execute("DROP TABLE IF EXISTS experiments CASCADE")
        # print("   ✅ All tables dropped")
        
        # # Recreate experiments table
        # print("\n4️⃣ Recreating tables with BYTEA schema...")
        # cursor.execute("""
        #     CREATE TABLE experiments (
        #         session_id TEXT PRIMARY KEY,
        #         parent_session_id TEXT,
        #         graph_topology TEXT,
        #         color_preview TEXT,
        #         timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        #         cwd TEXT,
        #         command TEXT,
        #         environment TEXT,
        #         code_hash TEXT,
        #         name TEXT,
        #         success TEXT CHECK (success IN ('', 'Satisfactory', 'Failed')),
        #         notes TEXT,
        #         log TEXT,
        #         FOREIGN KEY (parent_session_id) REFERENCES experiments (session_id),
        #         UNIQUE (parent_session_id, name)
        #     )
        # """)
        # print("   ✅ Experiments table created")
        
        # # Recreate llm_calls table with BYTEA output (THE CRITICAL FIX!)
        # cursor.execute("""
        #     CREATE TABLE llm_calls (
        #         session_id TEXT,
        #         node_id TEXT,
        #         input BYTEA,
        #         input_hash TEXT,
        #         input_overwrite BYTEA,
        #         output BYTEA,
        #         color TEXT,
        #         label TEXT,
        #         api_type TEXT,
        #         timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        #         PRIMARY KEY (session_id, node_id),
        #         FOREIGN KEY (session_id) REFERENCES experiments (session_id)
        #     )
        # """)
        # print("   ✅ LLM calls table created with BYTEA output column")
        
        # # Recreate attachments table
        # cursor.execute("""
        #     CREATE TABLE attachments (
        #         file_id TEXT PRIMARY KEY,
        #         session_id TEXT,
        #         line_no INTEGER,
        #         content_hash TEXT,
        #         file_path TEXT,
        #         taint TEXT,
        #         FOREIGN KEY (session_id) REFERENCES experiments (session_id)
        #     )
        # """)
        # print("   ✅ Attachments table created")
        
        # # Create indexes
        # cursor.execute("CREATE INDEX attachments_content_hash_idx ON attachments(content_hash)")
        # cursor.execute("CREATE INDEX original_input_lookup ON llm_calls(session_id, input_hash)")
        # cursor.execute("CREATE INDEX experiments_timestamp_idx ON experiments(timestamp DESC)")
        # print("   ✅ Indexes created")
        
        # # Verify the schema fix
        # cursor.execute("""
        #     SELECT data_type 
        #     FROM information_schema.columns 
        #     WHERE table_name = 'llm_calls' AND column_name = 'output'
        # """)
        # output_type = cursor.fetchone()[0]
        
        print(f"\n🎉 SUCCESS! Schema completely rebuilt")
        print(f"   ✅ Output column is now: {output_type}")
        print(f"   ✅ All tables recreated cleanly")
        print(f"   ✅ All locks cleared")
        
        if output_type == 'bytea':
            print(f"\n✨ The fix is complete! Now:")
            print(f"   1. Your original script should work without errors")
            print(f"   2. Revert the cache_manager.py change (use dill.loads directly)")
            print(f"   3. The web app should show clear data")
        else:
            print(f"   ⚠️  Output column type is unexpected: {output_type}")
            
        fresh_conn.close()
        
    except Exception as e:
        print(f"❌ Emergency reset failed: {e}")
        print(f"   Error type: {type(e).__name__}")
        
        if "timeout" in str(e).lower():
            print("   → Connection still timing out. Database may be overloaded.")
        elif "permission" in str(e).lower():
            print("   → Permission denied. Check database user privileges.")
        else:
            print(f"   → Unexpected error: {e}")
            
else:
    print("❌ No DATABASE_URL found")

🚨 Creating fresh connection and performing emergency reset...
1️⃣ Establishing fresh connection...
   ✅ Fresh connection established

2️⃣ Killing hanging sessions...
   ✅ Terminated 2 hanging sessions

3️⃣ Force dropping tables...
   ✅ All tables dropped

4️⃣ Recreating tables with BYTEA schema...
   ✅ Experiments table created
   ✅ LLM calls table created with BYTEA output column
   ✅ Attachments table created
   ✅ Indexes created

🎉 SUCCESS! Schema completely rebuilt
   ✅ Output column is now: bytea
   ✅ All tables recreated cleanly
   ✅ All locks cleared

✨ The fix is complete! Now:
   1. Your original script should work without errors
   2. Revert the cache_manager.py change (use dill.loads directly)
   3. The web app should show clear data


In [5]:
# Verify the schema change was successful
final_schema = db.query_all(
    """
    SELECT column_name, data_type, is_nullable
    FROM information_schema.columns 
    WHERE table_name = 'llm_calls' 
    ORDER BY ordinal_position
    """
)

print("Final llm_calls table schema:")
for col in final_schema:
    status = ""
    if col['column_name'] == 'output':
        if col['data_type'] == 'bytea':
            status = " ✅"
        else:
            status = " ❌"
    print(f"  {col['column_name']}: {col['data_type']} ({'NULL' if col['is_nullable'] == 'YES' else 'NOT NULL'}){status}")

# Test that we can insert and retrieve binary data
print("\nTesting binary data storage...")
try:
    import dill
    test_data = {"test": "data", "number": 42}
    test_pickle = dill.dumps(test_data)
    
    # This should work now without error
    print(f"✅ Successfully created pickle data: {len(test_pickle)} bytes")
    print(f"✅ Can load pickle back: {dill.loads(test_pickle)}")
    print("Migration appears successful!")
    
except Exception as e:
    print(f"❌ Error with pickle test: {e}")

2025-11-17 02:34:14,429 - ACO - DEBUG - Database schema initialized
2025-11-17 02:34:14,430 - ACO - INFO - Initialized PostgreSQL connection to workflow-postgres.cm14iy6021bi.us-east-1.rds.amazonaws.com


Final llm_calls table schema:
  session_id: text (NOT NULL)
  node_id: text (NOT NULL)
  input: bytea (NULL)
  input_hash: text (NULL)
  input_overwrite: bytea (NULL)
  output: bytea (NULL) ✅
  color: text (NULL)
  label: text (NULL)
  api_type: text (NULL)
  timestamp: timestamp without time zone (NULL)

Testing binary data storage...
✅ Successfully created pickle data: 41 bytes
✅ Can load pickle back: {'test': 'data', 'number': 42}
Migration appears successful!
